In [1]:
import yfinance as yf
import pandas as pd
import pandas_ta as ta


In [5]:
ticker_list = [
    "NVDA", "MSFT", "AAPL", "AMZN", "GOOGL", "META", "TSLA", "JPM", "WMT",
    "LLY", "V", "ORCL", "NFLX", "MA", "XOM", "UNH", "BAC", "PFE", "ABBV",
    "CSCO", "PEP", "MRK", "COST", "MCD", "AVGO", "NKE", "CRM", "HON", "WFC",
    "C", "TXN", "ABT", "ACN", "MDT", "PM", "BMY", "VZ", "LIN", "LOW",
    "UPS", "RTX", "DHR", "UNP", "TMO", "MS", "SCHW", "NEE", "CVX", "PNC"
]



In [6]:
def process_stock(ticker):
    Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
    
    Data.columns = Data.columns.droplevel(0)
    # Data.head(5)
    
    ## Add Useful KPIs
    
    Data['RSI']=ta.rsi(Data["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    Data['Sector']=info.get("sector")
    Data['Industry']=info.get("industry")
    
    MACD=ta.macd(Data["Close"], fast=12, slow=26, signal=9)
    Data=pd.concat([Data,MACD],axis=1)
    
    Data.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    Data["EMA_20"] = ta.ema(Data["Close"], length=20)
    Data["EMA_50"] = ta.ema(Data["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(Data["Close"], length=20)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, bbands], axis=1)
    
    # Optional rename (if needed)
    Data.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Join it with main DataFrame
    Data = pd.concat([Data, adx_df], axis=1)
    
    # Optional rename (for clarity)
    Data.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    Data["ATR_14"] = ta.atr(Data["High"], Data["Low"], Data["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(Data["High"], Data["Low"], Data["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    Data = pd.concat([Data, stoch_df], axis=1)
    
    # Optional rename
    Data.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    Data["CCI_20"] = ta.cci(Data["High"], Data["Low"], Data["Close"], length=20)
    
    # Data.to_csv("test.csv")
    
    Data.reset_index(inplace=True)
    
    ## Feature Engineering
    
    Data['Volume1']=Data['Volume'].shift(1)
    Data['Volume_Inc']=(Data['Volume']-Data['Volume1'])/Data['Volume1']
    
    Data['EMA_20_Price']=(Data['Close']-Data['EMA_20'])/Data['EMA_20']
    Data['EMA_50_Price']=(Data['Close']-Data['EMA_50'])/Data['EMA_50']
    
    Data['Stoch_k_D']=(Data['Stoch_%K']-Data['Stoch_%D'])
    
    Data['Close1']=Data['Close'].shift(10)
    Data['Close_Inc']=(Data['Close1']-Data['Close'])/Data['Close']
    
    # Data.head(50)
    
    def Buy_Variable(row):
        if row['Close_Inc']>=0.04:
            return 1
        else:
            return 0
    
    Data['Buy']=Data.apply(Buy_Variable,axis=1)
    
    ## Data Clean
    
    Clean_Data=Data[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D', 'Buy']]
    
    Clean_Data=Clean_Data.dropna(subset=['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price'])
    
    # Clean_Data.head(10)
    
    ## Visualize
    
    import seaborn as sns
    import matplotlib.pyplot as plt
    
    features = ["RSI", "MACD_Histogram", "BB_Percent", "ADX", "ATR_14", 
                "Stoch_%K", "CCI_20", "EMA_20_Price", "Volume_Inc"]
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 1], x=feature, label="Buy = 1", shade=True)
    #     sns.kdeplot(data=Clean_Data[Clean_Data["Buy"] == 0], x=feature, label="Buy = 0", shade=True)
    #     plt.title(f"{feature} vs Buy")
    #     plt.legend()
    #     plt.tight_layout()
    #     plt.show()
    
    
    # for feature in features:
    #     plt.figure(figsize=(6, 3))
    #     sns.boxplot(data=Clean_Data, x="Buy", y=feature)
    #     plt.title(f"{feature} by Buy value")
    #     plt.tight_layout()
    #     plt.show()
    
    
    Clean_Data.drop(columns="Volume_Inc",inplace=True)
    
    # # Step 1: Compute correlation matrix
    # corr_matrix = Clean_Data.corr()
    
    # # Step 2: Set up the heatmap
    # plt.figure(figsize=(12, 8))
    # sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", linewidths=0.5)
    # plt.title("Correlation Heatmap")
    # plt.tight_layout()
    # plt.show()
    
    # Clean_Data.head(5)
    
    ## Model Build
    
    from sklearn.preprocessing import StandardScaler
    
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(Clean_Data.drop("Buy", axis=1))
    
    
    
    # X_scaled
    
    from sklearn.model_selection import train_test_split
    y=Clean_Data['Buy']
    
    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.2, random_state=42, stratify=y
    )
    
    # ### Logisitic
    
    # from sklearn.linear_model import LogisticRegression
    
    # model=LogisticRegression()
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # from sklearn.metrics import classification_report,roc_auc_score
    # print("🔹 Logistic Regression:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    # ### Random Forest
    
    # from sklearn.ensemble import RandomForestClassifier
    
    # model=RandomForestClassifier(random_state=42)
    # model.fit(X_train, y_train)
    # y=model.predict(X_test)
    # y_proba = model.predict_proba(X_test)[:, 1]
    # feature_importances = model.feature_importances_
    # features = Clean_Data.drop("Buy", axis=1).columns
    # # Combine into a DataFrame
    # F_Imp = pd.DataFrame({
    #     "Feature": features,
    #     "Importance": feature_importances
    # }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    # from sklearn.metrics import classification_report
    # print("🔹 RandomForestClassifier:")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    ### XG Boost
    
    from xgboost import XGBClassifier
    model=XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
    model.fit(X_train, y_train)
    y=model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]
    feature_importances = model.feature_importances_
    features = Clean_Data.drop("Buy", axis=1).columns
    # Combine into a DataFrame
    F_Imp = pd.DataFrame({
        "Feature": features,
        "Importance": feature_importances
    }).sort_values("Importance", ascending=False)
    # print(F_Imp)
    from sklearn.metrics import classification_report
    # print("🔹 XGBClassifier: ")
    # print(classification_report(y_test, y))
    # print(f"ROC AUC Score: {roc_auc_score(y_test, y_proba):.4f}")
    
    
    
    data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
    data_Test.columns = data_Test.columns.droplevel(0)
    
    data_Test['RSI']=ta.rsi(data_Test["Close"], length=14)
    
    info = yf.Ticker(ticker).info
    data_Test['Sector']=info.get("sector")
    data_Test['Industry']=info.get("industry")
    
    MACD=ta.macd(data_Test["Close"], fast=12, slow=26, signal=9)
    data_Test=pd.concat([data_Test,MACD],axis=1)
    
    data_Test.rename(columns={
        "MACD_12_26_9": "MACD_Line",
        "MACDs_12_26_9": "MACD_Signal",
        "MACDh_12_26_9": "MACD_Histogram"
    }, inplace=True)
    
    
    data_Test["EMA_20"] = ta.ema(data_Test["Close"], length=20)
    data_Test["EMA_50"] = ta.ema(data_Test["Close"], length=50)
    
    # Add Bollinger Bands (20-day default)
    bbands = ta.bbands(data_Test["Close"], length=20)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, bbands], axis=1)
    
    # Optional rename (if needed)
    data_Test.rename(columns={
        "BBL_20_2.0": "BB_Lower",
        "BBM_20_2.0": "BB_Middle",
        "BBU_20_2.0": "BB_Upper",
        "BBB_20_2.0": "BB_Bandwidth",
        "BBP_20_2.0": "BB_Percent"
    }, inplace=True)
    
    # Add ADX (default 14-period)
    adx_df = ta.adx(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Join it with main DataFrame
    data_Test = pd.concat([data_Test, adx_df], axis=1)
    
    # Optional rename (for clarity)
    data_Test.rename(columns={
        "ADX_14": "ADX",
        "DMP_14": "DI_Plus",
        "DMN_14": "DI_Minus"
    }, inplace=True)
    
    # Add ATR (default length=14)
    data_Test["ATR_14"] = ta.atr(data_Test["High"], data_Test["Low"], data_Test["Close"], length=14)
    
    # Add Stochastic Oscillator (default k=14, d=3)
    stoch_df = ta.stoch(data_Test["High"], data_Test["Low"], data_Test["Close"], k=14, d=3)
    
    # Combine with main DataFrame
    data_Test = pd.concat([data_Test, stoch_df], axis=1)
    
    # Optional rename
    data_Test.rename(columns={
        "STOCHk_14_3_3": "Stoch_%K",
        "STOCHd_14_3_3": "Stoch_%D"
    }, inplace=True)
    
    
    
    # Add CCI (default period = 20)
    data_Test["CCI_20"] = ta.cci(data_Test["High"], data_Test["Low"], data_Test["Close"], length=20)
    
    data_Test['Volume1']=data_Test['Volume'].shift(1)
    data_Test['Volume_Inc']=(data_Test['Volume']-data_Test['Volume1'])/data_Test['Volume1']
    
    data_Test['EMA_20_Price']=(data_Test['Close']-data_Test['EMA_20'])/data_Test['EMA_20']
    data_Test['EMA_50_Price']=(data_Test['Close']-data_Test['EMA_50'])/data_Test['EMA_50']
    
    data_Test['Stoch_k_D']=(data_Test['Stoch_%K']-data_Test['Stoch_%D'])
    
    data_Test['Close1']=data_Test['Close'].shift(10)
    data_Test['Close_Inc']=(data_Test['Close1']-data_Test['Close'])/data_Test['Close']
    
    data_Test.head(50)
    
    def Buy_Variable(row):
        if row['Close_Inc']>=0.03:
            return 1
        else:
            return 0
    
    data_Test['Buy']=data_Test.apply(Buy_Variable,axis=1)
    
    data_Test.iloc[-2:-1]
    
    Clean_Data_Test=data_Test[['RSI', 'MACD_Histogram','BB_Percent','ADX','ATR_14', 'Stoch_%K',
           'Stoch_%D', 'CCI_20','Volume_Inc', 'EMA_20_Price',
           'EMA_50_Price', 'Stoch_k_D', 'Buy']]
    
    # Step 1: Get feature columns (same as used during training)
    features = ['RSI', 'MACD_Histogram', 'BB_Percent', 'ADX', 'ATR_14',
                'Stoch_%K', 'Stoch_%D', 'CCI_20', 'EMA_20_Price', 'EMA_50_Price', 'Stoch_k_D']
    
    # Step 2: Extract today's row (last row in the test table)
    today_row = Clean_Data_Test[features].iloc[-1:]  # Keep it as DataFrame
    
    # Step 3: Apply the same scaler used during training
    today_scaled = scaler.transform(today_row)
    
    # Step 4: Predict using the trained model
    buy_prediction = model.predict(today_scaled)[0]
    print(today_scaled)
    buy_probability = model.predict_proba(today_scaled)[0][1]
    
    # Step 5: Print result
    # print("🔮 Buy Prediction:", "Yes" if buy_prediction == 1 else "No")
    # print(f"📊 Confidence: {buy_probability:.2%}")
    return {
            "Ticker": ticker,
            "Buy": "Yes" if buy_prediction == 1 else "No",
            "Confidence": round(buy_probability * 100, 2)
        }
    
    


In [7]:
results = []

for ticker in ticker_list:
    result = process_stock(ticker)
    results.append(result)

final_df = pd.DataFrame(results)


print(final_df)


C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:01:41] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[1.58889987 0.35463583 1.01084066 2.36542546 0.69825371 1.27853768
  1.21371322 0.83095857 0.77556621 1.20298207 0.35002582]]


C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:01:43] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 1.44870382 -0.62970282  0.78501083  2.76138947 -0.10133964  0.96158093
   1.04122299  0.67078054  0.63165393  1.51900722 -0.16220538]]


C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:01:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


...

[[ 0.66556361  0.32894582  0.91431611 -1.37664228  1.33695018  0.97139484
   0.98853419  0.70228413  0.83178973  1.03700413  0.0348578 ]]


C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:03:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 1.56540739  0.78367969  1.46965633  2.01041446 -0.04633842  1.36907583
   0.95613066  1.42996196  1.15013208  1.22809942  1.46856907]]


C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:2: FutureWarning: YF.download() has changed argument auto_adjust default to True
  Data=yf.download(tickers=ticker,period="5y",interval="1d",group_by=ticker)
[*********************100%***********************]  1 of 1 completed
C:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\training.py:183: UserWarning: [02:03:55] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\PC\AppData\Local\Temp\ipykernel_5920\1505815251.py:229: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data_Test = yf.download(tickers=ticker, period="60d", interval="1d", group_by="ticker")
[*********************100%***********************]  1 of 1 completed


[[ 2.07928423  1.1361417   0.96049222  2.05552609 -0.51245961  1.19977821
   1.35907333  0.8565      1.29434195  1.61895318 -0.39877139]]
   Ticker  Buy  Confidence
0    NVDA   No        0.00
1    MSFT   No        0.01
2    AAPL   No        0.00
3    AMZN   No        0.01
4   GOOGL   No        0.01
5    META   No        0.01
6    TSLA  Yes       92.96
7     JPM   No        0.03
8     WMT   No        8.39
9     LLY   No        0.01
10      V   No        0.02
11   ORCL   No        0.05
12   NFLX   No        0.54
13     MA   No        0.01
14    XOM   No        0.01
15    UNH   No        0.04
16    BAC   No        0.05
17    PFE   No        0.00
18   ABBV   No        0.01
19   CSCO   No        0.00
20    PEP   No        0.02
21    MRK   No        0.01
22   COST   No        0.09
23    MCD   No        0.02
24   AVGO   No        0.03
25    NKE   No        0.01
26    CRM   No        3.11
27    HON   No        0.01
28    WFC   No        0.00
29      C   No        0.00
30    TXN   No        0.0

In [9]:
from datetime import date

today = date.today().strftime("%Y-%m-%d")  # e.g., "2025-07-08"
filename = f"{today}_Screener_USA.csv"

final_df.to_csv(filename, index=False)
